In [1]:
import torch

# 检查 MPS 是否可用
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Using MPS backend on Apple Silicon.")
else:
    device = torch.device("cpu")
    print("MPS backend not available, using CPU instead.")


Using MPS backend on Apple Silicon.


2024-11-08 22:37:23.369 python[54418:454522] getMetalPluginClassForService: Failed to find bundle for accelerator bundle named: AGXMetalA12 errno: 0


In [2]:
import os
import time
import logging
import pickle
import tqdm
import json
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np

In [3]:
BASE_DIR = '/Users/jessie/Documents/Projects/Cusanus_Topic_Modeling'

In [4]:
# 生成唯一实验ID
experiment_id = f"bertopic_experiment_{int(time.time())}"
experiment_dir = os.path.join(BASE_DIR, 'experiments', experiment_id)

# 创建实验目录
if not os.path.exists(experiment_dir):
    os.makedirs(experiment_dir)

# 获取日志文件路径
log_file_path = os.path.join(experiment_dir, f"bertopic_experiment_{experiment_id}.log")

# 配置日志记录
logger = logging.getLogger()
if logger.hasHandlers():
    logger.handlers.clear()
logger.setLevel(logging.INFO)
formatter = logging.Formatter("%(asctime)s - %(levelname)s - %(message)s")
file_handler = logging.FileHandler(log_file_path)
file_handler.setFormatter(formatter)
logger.addHandler(file_handler)
console_handler = logging.StreamHandler()
console_handler.setFormatter(formatter)
logger.addHandler(console_handler)

logger.info(f"启动实验 {experiment_id}，日志记录到 {log_file_path}")
print(f"启动实验 {experiment_id}，日志记录到 {log_file_path}")

2024-11-08 22:37:29,181 - INFO - 启动实验 bertopic_experiment_1731101849，日志记录到 /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/experiments/bertopic_experiment_1731101849/bertopic_experiment_bertopic_experiment_1731101849.log


启动实验 bertopic_experiment_1731101849，日志记录到 /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/experiments/bertopic_experiment_1731101849/bertopic_experiment_bertopic_experiment_1731101849.log


In [5]:
# 实验配置
experiment_config = {
    "parameters": {
        "n_gram_range": (1, 2),  # 捕捉更多的上下文短语
        "min_topic_size": 2,  # 增大最小主题大小以减少噪声
        "nr_topics": "auto",  # 自动确定主题数量
        "umap_params": {
            "n_neighbors": 10,  # 增加邻居数使得降维更平滑
            "min_dist": 0.1,  # 增大最小距离使主题更分离
            "n_components": 2,  # 增大维度以保留更多特征信息
            "random_state": 42  # 确保实验可重复
        }
    }
}

config_path = os.path.join(experiment_dir, 'config.json')
with open(config_path, 'w') as config_file:
    json.dump(experiment_config, config_file, indent=4)

logger.info(f"实验配置已保存到 {config_path}")
print(f"实验配置已保存到 {config_path}")

2024-11-08 22:37:29,210 - INFO - 实验配置已保存到 /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/experiments/bertopic_experiment_1731101849/config.json


实验配置已保存到 /Users/jessie/Documents/Projects/Cusanus_Topic_Modeling/experiments/bertopic_experiment_1731101849/config.json


In [6]:
# 加载模型和分词器
model_name = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

# 将模型移动到 MPS 设备
model.to(device)


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(119547, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [7]:
# 定义嵌入生成函数
def get_mbert_embeddings(texts):
    tokens = tokenizer(texts, padding=True, truncation=True, return_tensors="pt", max_length=256).to(device)
    with torch.no_grad():
        # 将计算过程放到 MPS 上
        embeddings = model(**tokens).last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()  # 返回到 CPU 上，以便后续处理

In [8]:
# 测试运行 MPS 加速
texts = ["Gallia est omnis divisa in partes tres.", "Alea iacta est."]
embeddings = get_mbert_embeddings(texts)
print("Embeddings shape:", embeddings.shape)


Embeddings shape: (2, 768)


In [9]:
# 创建 CountVectorizer 和 UMAP
vectorizer_model = CountVectorizer(
    ngram_range=experiment_config["parameters"]["n_gram_range"],
    token_pattern=r"(?u)\b\w+\b"
)
custom_umap = UMAP(
    n_neighbors=experiment_config["parameters"]["umap_params"]["n_neighbors"],
    min_dist=experiment_config["parameters"]["umap_params"]["min_dist"],
    n_components=experiment_config["parameters"]["umap_params"]["n_components"],
    random_state=experiment_config["parameters"]["umap_params"]["random_state"]
)

In [10]:
from tqdm import tqdm

# 加载测试集文档
testset_dir = os.path.join(BASE_DIR, 'data/berttest')
documents = []

logger.info("加载测试集数据...")
test_files = [f for f in os.listdir(testset_dir) if f.endswith('.txt')]
for test_file in tqdm(test_files, desc="Loading testset files"):
    file_path = os.path.join(testset_dir, test_file)
    with open(file_path, 'r', encoding='utf-8') as file:
        documents.append(file.read())

logger.info(f"加载了 {len(documents)} 个文档用于 BERTopic 实验。")
print(documents[:5])


2024-11-08 22:37:31,888 - INFO - 加载测试集数据...
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Loading testset files: 100%|██████████| 5/5 [00:00<00:00, 2666.78it/s]
2024-11-08 22:37:31,929 - INFO - 加载了 5 个文档用于 BERTopic 实验。


['commemoratio lucae clementinis dominus sanctus reliquiae veneratio sanc¬tor vir honorabilis christus dilectus fra¬tres insto devotio tempus tempus piger cor redeo debeo salus cor inflammo cari¬tas ardor sedulus meditatio pensaris excusatio sperno eloquen¬tiae fuco scientia vito ineptissi¬mus sperno allegatio simplex quoad conor cunctipotentis gratia mens via salutaris meditatio incitaris meditatio verbum exprimo instituo inardescat ignis spirital affectio qui1 anima sacro passio tempus divinus amor gaudium vitalis incalesco eapropter dilectus utor doctrina urbanus reliquiae veneratio sancio dominus exordior intentio laus sacrosanctus sacramentum novus lex utor exordior commemoratio epilogus tres tres tempus magnus ecclesiastici vir superus studium meditaris debeo memoria acerbus passio salvator salutifer eucharistiae sacramentum dies pius testamentum legatus necnon tertius sacramentum confectio cum2 quattuor effectus materia forma per¬ceptio praeho verbum christus salvator ostendo th

In [ ]:
# 初始化 BERTopic 模型，不设置 embedding_model
topic_model = BERTopic(
    embedding_model=None,  # 不使用默认的嵌入模型
    vectorizer_model=vectorizer_model,
    umap_model=custom_umap,
    min_topic_size=experiment_config["parameters"]["min_topic_size"],
    nr_topics=experiment_config["parameters"]["nr_topics"],
    language=None  # 禁用语言特定的嵌入
)

# 手动生成嵌入
print("Generating embeddings for documents...")
custom_embeddings = get_mbert_embeddings(documents)

# 使用手动生成的嵌入进行主题模型训练
if documents:
    print("Starting BERTopic model training...")
    logger.info("开始训练 BERTopic 模型...")
    
    # 将生成的 numpy 格式的自定义嵌入传入 fit_transform
    topics, probabilities = topic_model.fit_transform(documents, embeddings=custom_embeddings)
    logger.info("BERTopic 模型训练成功。")
    print("BERTopic model training completed.")
    
    # 可视化和保存结果
    topics_info = topic_model.get_topic_info()
    for topic_num in topics_info['Topic'][:10]:  # 输出前 10 个主题
        if topic_num != -1:
            words_weights = topic_model.get_topic(topic_num)
            words_str = ', '.join([word for word, _ in words_weights])
            print(f"主题 {topic_num}: {words_str}")
            logger.info(f"主题 {topic_num}: {words_str}")

    # 保存文档的主题分配结果
    document_topic_data = []
    for doc_idx, (topic, prob) in enumerate(zip(topics, probabilities)):
        document_name = os.path.basename(test_files[doc_idx])
        document_topic_data.append([document_name, topic, prob])

    df_document_topics = pd.DataFrame(document_topic_data, columns=["Document", "Assigned Topic", "Probability"])
    document_topics_csv_path = os.path.join(experiment_dir, 'bertopic_document_topic_distribution.csv')
    df_document_topics.to_csv(document_topics_csv_path, index=False)
    logger.info(f"每个文档的主题分配结果已保存至 {document_topics_csv_path}。")


Generating embeddings for documents...


2024-11-08 22:37:32,222 - INFO - 开始训练 BERTopic 模型...


Starting BERTopic model training...
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
# 保存可视化图表为 HTML 文件
try:
    topics_fig_path = os.path.join(experiment_dir, 'bertopic_topics.html')
    fig = topic_model.visualize_topics()
    fig.write_html(topics_fig_path)
    logger.info(f"主题可视化图表已保存至 {topics_fig_path}")

    # 生成并保存层次聚类图表
    fig_hierarchy = topic_model.visualize_hierarchy()
    fig_hierarchy.show()
    hierarchy_fig_path = os.path.join(experiment_dir, 'bertopic_hierarchy.html')
    fig_hierarchy.write_html(hierarchy_fig_path)
    logger.info(f"层次聚类图表已保存至 {hierarchy_fig_path}")
except Exception as e:
    logger.error(f"保存可视化图表时发生错误: {e}")
    raise